In [2]:
from auto_login import get_curr_path,get_credentials,auto_login,generate_access_token
import os 
import logging 
from kiteconnect import KiteConnect
import datetime as dt
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 
import ta

In [4]:
folder_name = 'api_keys'
curr_dir = get_curr_path(folder_name)

file_name = 'credentials.txt'
api_key,api_secret,user_name,pwd,totp_key = get_credentials(curr_dir,file_name)

request_token = auto_login(api_key,user_name,pwd,totp_key)

access_token = generate_access_token(request_token,api_key,api_secret)

In [5]:
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token=access_token)
instrument_dump = kite.instruments('NSE')
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv('NSE_instruments.csv',index=False)

In [6]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1

In [7]:
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

In [8]:
ohlc_day_df = fetchOHLC("INFY","day",30)

In [9]:
infy_df = ohlc_day_df.copy()

In [10]:
infy_df.to_csv('infy_day.csv')

In [12]:
infy_df.head()

,open,high,low,close,volume
date,,,,,
2024-05-13 00:00:00+05:30,1425.00,1425.00,1411.20,1423.45,3735954
2024-05-14 00:00:00+05:30,1426.40,1433.75,1418.05,1424.70,3463230
2024-05-15 00:00:00+05:30,1426.00,1431.80,1418.55,1419.95,4327544
2024-05-16 00:00:00+05:30,1433.95,1455.75,1427.00,1453.35,9249145
2024-05-17 00:00:00+05:30,1457.00,1457.00,1439.25,1444.30,7542860


In [13]:
infy_df = pd.read_csv('infy_day.csv')
infy_df.set_index("date",inplace=True)
infy_df

,open,high,low,close,volume
date,,,,,
2024-05-13 00:00:00+05:30,1425.00,1425.00,1411.20,1423.45,3735954
2024-05-14 00:00:00+05:30,1426.40,1433.75,1418.05,1424.70,3463230
2024-05-15 00:00:00+05:30,1426.00,1431.80,1418.55,1419.95,4327544
2024-05-16 00:00:00+05:30,1433.95,1455.75,1427.00,1453.35,9249145
2024-05-17 00:00:00+05:30,1457.00,1457.00,1439.25,1444.30,7542860
2024-05-18 00:00:00+05:30,1445.00,1450.00,1442.00,1443.65,318277
2024-05-21 00:00:00+05:30,1430.00,1444.70,1428.40,1434.15,6752663
2024-05-22 00:00:00+05:30,1428.75,1459.95,1428.00,1454.80,7173285
2024-05-23 00:00:00+05:30,1454.70,1478.15,1450.00,1472.40,8641428


In [ ]:
    # PP = (HIGHprev + LOWprev + CLOSEprev) / 3
    # R1 = 2 * PP - LOWprev
    # S1 = 2 * PP - HIGHprev
    # R2 = PP + (HIGHprev - LOWprev)
    # S2 = PP - (HIGHprev - LOWprev)
    # R3 = PP + 2 * (HIGHprev - LOWprev)
    # S3 = PP - 2 * (HIGHprev - LOWprev)
    # R4 = PP + 3 * (HIGHprev - LOWprev)
    # S4 = PP - 3 * (HIGHprev - LOWprev)

In [15]:
def compute_pivot_point(df):
    h = round(df.loc[df.index[-1],'high'],2)
    l = round(df.loc[df.index[-1],'low'],2) 
    c = round(df.loc[df.index[-1],'close'],2)
    p = round((h + l + c) / 3,2) 
    s1 = round((2*p) - h,2) 
    s2 = round(p - (h - l),2)
    s3 = round(p - (2*(h-l)),2)
    r1 = round((2 * p) - l,2) 
    r2 = round(p + (h - l),2)
    r3 = round(p + 2*(h - l),2)
    return (p,s1,s2,s3,r1,r2,r3)

In [16]:
pivot_point_levels = compute_pivot_point(infy_df)

In [17]:
pivot_point_levels

(1516.85, 1494.0, 1454.4, 1391.95, 1556.45, 1579.3, 1641.75)